In [2]:
import pandas as pd
import glob
import bamboolib
import plotly.express as px
import plotly.graph_objs as go

## Read files

In [3]:
def read_file_who(folder_path,type_name,page,skip_row):
    # get a list of all xlsx files in the folder
    df_all = []
    for (path,name) in zip(folder_path,type_name):
        file_list = glob.glob(path + '/*.xlsx')
        df_list = []
        # loop through each file and read the specified sheet
        for file_name in file_list:
            excel_file = pd.ExcelFile(file_name)
            if page == None:
                sheet_names = excel_file.sheet_names
            else:
                sheet_names = page
            # read the file and the specified sheet
            for sheet in sheet_names:
                if skip_row == None:
                    df = pd.read_excel(file_name, sheet_name=sheet)
                else:
                    df = pd.read_excel(file_name, sheet_name=sheet,skiprows=skip_row,header = 0)
                df['Option'] = sheet
                df_list.append(df)
                output_df = pd.concat(df_list)
        output_df['Type'] = name
        df_all.append(output_df)
    
    all_data_df = pd.concat(df_all)
    return all_data_df 




In [4]:
folder_path = ['raw/government_expenditure_on_routine_immunization',
    'raw/government_expenditure_on_vaccine_used_in_routine_immunization',
    'raw/total_expenditure_on_routine_immunization',
    'raw/total_expenditure_on_vaccine_used_in_routine_immunization']
type_name = ['gov_immunization','gov_vaccine','total_immunization','total_vaccine']

expenditure_df = read_file_who(folder_path,type_name,None,None)

coverage_df = read_file_who(['raw\coverage'],['coverage'],['Data'],None)

population_df = read_file_who(['raw\population'],['population'],['Estimates'],16)

gdp_df = read_file_who(['raw\GDP'],['current GDP'],['Data'],3)

land_df = read_file_who(['raw\land'],['land'],['SYB65_145_202209_Land'],1)


## Data Preparation

In [5]:
#Standardize the country name
def replace_country_names(df, replacements):
    for original, replacement in replacements.items():
        df.loc[df['Country'] == original, 'Country'] = replacement
    return df

replacements = {
    'Bolivia (Plurinational State of)': 'Bolivia',
    'Congo (The)': 'Congo',
    #'Democratic Republic of the Congo': 'Congo',
    'Czechia': 'Czech Republic',
    'Iran (Islamic Republic of)': 'Iran',
    'Lao People\'s Democratic Republic': 'Laos',
    'Lao People\'s Democratic Republic (the)': 'Laos',
    'North Macedonia': 'North Macedonia',
    'Republic of Macedonia': 'North Macedonia',
    'The former Yugoslav Republic of Macedonia': 'North Macedonia',
    'Netherlands': 'The Netherlands',
    'Saint Vincent and the Grenadines': 'Saint Vincent and the Grenadines',
    'Saint Vincent and The Grenadines': 'Saint Vincent and the Grenadines',
    'Sudan': 'Sudan',
    'South Sudan': 'South Sudan',
    'Syrian Arab Republic': 'Syria',
    'Syria': 'Syria',
    'Tanzania': 'Tanzania',
    'United Republic of Tanzania': 'Tanzania',
    'Turks and Caicos Islands': 'Turks and Caicos Islands',
    'Turks and Caicos Islands, The': 'Turks and Caicos Islands',
    'United Kingdom': 'United Kingdom',
    'England': 'United Kingdom',
    'Wales': 'United Kingdom',
    'Scotland': 'United Kingdom',
    'Northern Ireland': 'United Kingdom',
    'Venezuela (Bolivarian Republic of)': 'Venezuela',
    'Viet Nam': 'Vietnam',
    'Bolivia (Plurin. State of)':'Bolivia',
    'Côte d’Ivoire':"Côte d'Ivoire",
    'Dem. Rep. of the Congo':'Democratic Republic of the Congo',
    "Lao People's Dem. Rep.":'Laos',
    "Dem. People's Rep. Korea":'Democratic Peoples Republic of Korea',
    'State of Palestine' : 'West Bank and Gaza',
    'Russian Federation' :"Russia",
    'Türkiye':'Turkey',
    'United Rep. of Tanzania':'Tanzania',
    'Saint Vincent & Grenadines':'Saint Vincent and the Grenadines',
    'Venezuela (Boliv. Rep. of)' : 'Venezuela'
}



In [6]:
def get_missing_value_percentage(expenditure_detail_df,columns_to_group=['Country'], columns_to_filter=['total_immunization in USD', 'total_vaccine in USD'], threshold=0.3,more_than = True):
    # group the data by both 'Year' and 'Country'
    grouped = expenditure_detail_df.groupby(columns_to_group)
    
    # calculate the percentage of missing values for each column within each group
    nan_percentages = grouped.apply(lambda x: x.isna().mean())
    
    # filter the results to only show the specified columns
    filtered = nan_percentages[columns_to_filter]
    
    # filter the results to only show percentages above the threshold value
    if more_than == True:
        filtered = filtered[filtered > threshold].dropna()
    else:
        filtered = filtered[filtered < threshold].dropna()
    return filtered

### Land

In [7]:
land_df = land_df[land_df['Series'] == 'Land area (thousand hectares)']
# Get the latest year
latest_year = land_df['Year'].max()

# Create a new dataframe with the latest values for each country/region
country_area_df = land_df.loc[land_df['Year'] == latest_year, ["Unnamed: 1", 'Value']].copy()

# Rename the 'Region/Country/Area' column to 'Country'
country_area_df = country_area_df.rename(columns={"Unnamed: 1": 'Country','Value':'land_area'})
country_area_df = replace_country_names(country_area_df, replacements)

### Goverment expenditure on immunization

In [8]:
#expenditure_df = read_file_who(folder_path,type_name,None,None)
expenditure_df.columns = expenditure_df.columns.astype(str)
value_vars = expenditure_df.columns.drop(['ISO', 'Country', 'Region', 'Gavi / Income status', 'Option', 'Type'])
expenditure_melt_df = pd.melt(expenditure_df, id_vars=['ISO', 'Country', 'Region', 'Gavi / Income status', 'Option', 'Type'], 
                 value_vars=value_vars, 
                 var_name='Year', value_name='value')

expenditure_melt_df['Year'] = pd.to_datetime(expenditure_melt_df['Year'], format='%Y').dt.year
expenditure_df = expenditure_melt_df


In [9]:
gov_type = ['gov_immunization', 'gov_vaccine']
gov_list = ['in USD', 'in USD per capita', 'in USD per surviving infant', '% government expenditure']
start = 0
for j in gov_type:
    for i in gov_list:
        if start == 0:
            expenditure_detail_df = expenditure_df[(expenditure_df['Type'] == j) & (expenditure_df['Option'] == i)][['ISO', 'Country', 'Region', 'Gavi / Income status', 'Year', 'value']]
            expenditure_detail_df = expenditure_detail_df.rename(columns={'value': j + ' ' + i})
        else:
            df_2 = expenditure_df[(expenditure_df['Type'] == j) & (expenditure_df['Option'] == i)][['ISO', 'Country', 'Region', 'Gavi / Income status', 'Year', 'value']]
            df_2 = df_2.rename(columns={'value': j + ' ' + i})
            expenditure_detail_df = pd.merge(expenditure_detail_df, df_2, on=['ISO', 'Country', 'Region', 'Gavi / Income status', 'Year'], how='outer', suffixes=('', ''))
        start += 1 

total_type = ['total_immunization', 'total_vaccine']
total_list = ['in USD', 'in USD per capita', 'in USD per surviving infant']
for j in total_type:
    for i in total_list:
        df_2 = expenditure_df[(expenditure_df['Type'] == j) & (expenditure_df['Option'] == i)][['ISO', 'Country', 'Region', 'Gavi / Income status', 'Year', 'value']]
        df_2 = df_2.rename(columns={'value': j + ' ' + i})
        expenditure_detail_df = pd.merge(expenditure_detail_df, df_2, on=['ISO', 'Country', 'Region', 'Gavi / Income status', 'Year'], how='outer', suffixes=('', ''))

In [10]:
expenditure_detail_df = replace_country_names(expenditure_detail_df, replacements)
expenditure_detail_df['Year']= pd.to_datetime(expenditure_detail_df['Year'], format='%Y')

In [11]:
#expenditure_detail_filt_df = expenditure_detail_df[['Country','Year','Region','Gavi / Income status','total_immunization in USD per surviving infant','total_vaccine in USD per surviving infant']]
expenditure_detail_filt_df = expenditure_detail_df[['Country','Year','Region','Gavi / Income status','total_immunization in USD','total_vaccine in USD','total_immunization in USD per surviving infant','total_vaccine in USD per surviving infant']]

In [12]:
#expenditure_detail_filt_df = expenditure_detail_filt_df.rename(columns = {'total_immunization in USD per surviving infant':'total_immunization in USD', 'total_vaccine in USD per surviving infant':'total_vaccine in USD'})

### Immunization coverage

In [13]:
coverage_df = coverage_df.rename(columns={'NAME': 'Country','YEAR':'Year'})
coverage_df = coverage_df[coverage_df['COVERAGE_CATEGORY'].isin(['WUENIC'])]
coverage_df = coverage_df[~coverage_df['ANTIGEN'].isin(['YFV'])]
coverage_df = replace_country_names(coverage_df, replacements)
coverage_df['Year'] = pd.to_datetime(coverage_df['Year'], format='%Y')

In [14]:
# create a pivot table for each antigen
antigen_pivot = pd.pivot_table(coverage_df, index=['Country', 'Year'], 
                               columns='ANTIGEN',
                               values=['COVERAGE'], 
                               aggfunc='sum')

# flatten the column index
antigen_pivot.columns = [f'{col[1]}_{col[0]}' for col in antigen_pivot.columns]

# reset the index to make Country and Year columns
antigen_pivot = antigen_pivot.reset_index()

#rename
immunization_coverage_df = antigen_pivot

### World Population

In [15]:
#population_df = read_file_who(['raw\population'],['population'],['Estimates'],16)
population_df

cols = [i for i in range(100)]
total = population_df[cols].sum(axis=1)
cols = [i for i in range(1,6)]
total_below_5 = population_df[cols].sum(axis=1)
cols = [i for i in range(6,11)]
total_below_10 = population_df[cols].sum(axis=1)
cols = [i for i in range(10,16)]
total_below_15 = population_df[cols].sum(axis=1)
cols = [i for i in range(16,21)]
total_below_20 = population_df[cols].sum(axis=1)
cols = [i for i in range(21,26)]
total_below_25 = population_df[cols].sum(axis=1)
cols = [i for i in range(0,1)]
total_below_1 = population_df[cols].sum(axis=1)

# add the 'total' column to the DataFrame
population_df['total_population'] = total
population_df['total_population_1-5'] = total_below_5
population_df['total_population_6-10'] = total_below_10
population_df['total_population_11-15'] = total_below_15
population_df['total_population_16-20'] = total_below_20
population_df['total_population_21-25'] = total_below_25
population_df['total_population_0-1'] = total_below_1
population_df['total_population'] = population_df['total_population'] + population_df['100+']
population_df = population_df.rename(columns={'Region, subregion, country or area *': 'Country'})
population_df['Year'] = pd.to_datetime(population_df['Year'], format='%Y')
population_df = replace_country_names(population_df, replacements)
population_df.columns = [str(column) for column in population_df.columns]

In [16]:
list = ['total_population','total_population_0-1','total_population_1-5', 'total_population_6-10',
       'total_population_11-15', 'total_population_16-20']
for i in list:
    population_df[i] = pd.to_numeric(population_df[i], downcast='float', errors='coerce')

In [17]:
population_filt_df = population_df[['Country','Year','total_population','total_population_0-1','total_population_1-5','total_population_6-10',
       'total_population_11-15', 'total_population_16-20']]

### GDP

In [18]:
#gdp_df = read_file_who(['raw\GDP'],['current GDP'],['Data'],3)

gdp_df.columns = gdp_df.columns.astype(str)
value_vars = gdp_df.columns.drop(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', 'Option', 'Type'])
gdp_melt_df = pd.melt(gdp_df, id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', 'Option', 'Type'], 
                 value_vars=value_vars, 
                 var_name='Year', value_name='GDP')
gdp_df = gdp_melt_df.rename(columns={'Country Name':'Country'})
gdp_df['GDP_Million'] = gdp_df['GDP']/1000000
gdp_df['Year'] = pd.to_datetime(gdp_df['Year'], format='%Y')
gdp_df = replace_country_names(gdp_df, replacements)

#rename
gdp_filt_df = gdp_df

### Merge Data

In [127]:
merged_df = pd.merge(expenditure_detail_filt_df[['Country', 'Region', 'Gavi / Income status', 'Year',
       'total_vaccine in USD',  'total_immunization in USD','total_immunization in USD per surviving infant','total_vaccine in USD per surviving infant'
     ]].drop_duplicates(),immunization_coverage_df.drop_duplicates(), on = ['Country','Year'], how = 'left')
merged_df = pd.merge(merged_df.drop_duplicates(),gdp_filt_df[['Country','GDP_Million','Year']].drop_duplicates(), on = ['Country','Year'], how = 'left')
merged_df = pd.merge(merged_df.drop_duplicates(),population_filt_df[['Country','Year','total_population','total_population_0-1','total_population_1-5', 'total_population_6-10',
       'total_population_11-15', 'total_population_16-20']].drop_duplicates(),on = ['Country','Year'],how ='left')
merged_df = pd.merge(merged_df.drop_duplicates(),country_area_df[['Country','land_area']].drop_duplicates(),on = ['Country'],how ='left')

### Clean data

##### Exclude the rich country (high income)

In [128]:
merged_df = merged_df[merged_df['Gavi / Income status'] != 'High income countries']

##### Exclude coverage that is 0

In [129]:
cols = ['BCG_COVERAGE', 'DTPCV1_COVERAGE', 'DTPCV3_COVERAGE', 'HEPB3_COVERAGE', 'HEPB_BD_COVERAGE', 'HIB3_COVERAGE', 'MCV1_COVERAGE', 'MCV2_COVERAGE', 'PCV3_COVERAGE', 'POL3_COVERAGE', 'RCV1_COVERAGE', 'ROTAC_COVERAGE']

for col in cols:
    below_one = merged_df[col][merged_df[col] < 1]
    print(f"{col}: {len(below_one)} values below 1")


BCG_COVERAGE: 0 values below 1
DTPCV1_COVERAGE: 0 values below 1
DTPCV3_COVERAGE: 0 values below 1
HEPB3_COVERAGE: 55 values below 1
HEPB_BD_COVERAGE: 1164 values below 1
HIB3_COVERAGE: 336 values below 1
MCV1_COVERAGE: 0 values below 1
MCV2_COVERAGE: 692 values below 1
PCV3_COVERAGE: 1038 values below 1
POL3_COVERAGE: 0 values below 1
RCV1_COVERAGE: 782 values below 1
ROTAC_COVERAGE: 1469 values below 1


In [130]:
coverage_list = ['BCG_COVERAGE', 'DTPCV1_COVERAGE', 'DTPCV3_COVERAGE', 'HEPB3_COVERAGE', 'HEPB_BD_COVERAGE', 'HIB3_COVERAGE', 'MCV1_COVERAGE', 'MCV2_COVERAGE', 'PCV3_COVERAGE', 'POL3_COVERAGE', 'RCV1_COVERAGE', 'ROTAC_COVERAGE']

# for i in coverage_list:
#     merged_df = merged_df[merged_df[i] > 0]

merged_df = merged_df.drop(['MCV2X2_COVERAGE','IPV1_COVERAGE'] , axis = 1)

##### Create new column to calculate the cost of vaccine delivery

In [131]:
merged_df['Year'] = pd.to_datetime(merged_df['Year'], format='%Y')
merged_df['total_vaccine_delivery_cost in USD'] = merged_df['total_immunization in USD'] -merged_df['total_vaccine in USD']


In [132]:
#change the data into Million format
merged_df['Vaccine USD Mil'] = merged_df['total_vaccine in USD'] / 1000000
merged_df['Immunization USD Mil'] = merged_df['total_immunization in USD'] / 1000000
merged_df['Vaccine Delivery Cost USD Mil'] = merged_df['total_vaccine_delivery_cost in USD'] / 1000000

In [133]:
# group the dataframe by country
grouped_df = merged_df.groupby('Country')

# Define a function to create the previous year columns in million
def create_prev_year_cols_million(col):
    return [f"{col}_prev_{i}_year_million" for i in range(1, 6)]

# Create new columns for 'total_vaccine in USD', 'total_immunization in USD', and 'total_vaccine_delivery_cost in USD', 
# for the value for previous year up until last 5 years, in million
for col in ['total_vaccine in USD', 'total_immunization in USD', 'total_vaccine_delivery_cost in USD']:
    prev_year_cols = create_prev_year_cols_million(col)
    for i, prev_year_col in enumerate(prev_year_cols):
        merged_df[prev_year_col] = grouped_df[col].apply(lambda x: x.shift(i+1) / 1000000)
        #merged_df[prev_year_col] = grouped_df[col].apply(lambda x: x.shift(i+1) )
        # Calculate the percentage change between the current year and the previous year
        curr_col = f"{col}_prev_1_year_million"
        pct_col = f"{col}_prev_{i+1}_year_pct_change"
        if curr_col in merged_df.columns and prev_year_col in merged_df.columns:
            merged_df[pct_col] = (merged_df[curr_col] - merged_df[prev_year_col]) / merged_df[prev_year_col]
        else:
            merged_df[pct_col] = float('nan')


In [134]:
# group the dataframe by country
grouped_df = merged_df.groupby('Country')

# Define a function to create the previous year columns in million
def create_prev_year_cols_million(col):
    return [f"{col}_prev_{i}_year_million" for i in range(1, 6)]

# Create new columns for 'total_vaccine in USD', 'total_immunization in USD', and 'total_vaccine_delivery_cost in USD', 
# for the value for previous year up until last 5 years, in million
for col in ['total_vaccine in USD', 'total_immunization in USD', 'total_vaccine_delivery_cost in USD']:
    prev_year_cols = create_prev_year_cols_million(col)
    for i, prev_year_col in enumerate(prev_year_cols):
        merged_df[prev_year_col] = grouped_df[col].apply(lambda x: x.shift(i+1) / 1000000)
        #merged_df[prev_year_col] = grouped_df[col].apply(lambda x: x.shift(i+1) )
        # Calculate the percentage change between the current year and the previous year
        curr_col = f"{col}_prev_1_year_million"
        pct_col = f"{col}_prev_{i+1}_year_pct_change"
        if curr_col in merged_df.columns and prev_year_col in merged_df.columns:
            merged_df[pct_col] = (merged_df[curr_col] - merged_df[prev_year_col]) / merged_df[prev_year_col]
        else:
            merged_df[pct_col] = float('nan')


In [135]:
merged_df['total_vaccine in USD_prev_1_year_pct_change'] = (merged_df['Vaccine USD Mil'] - merged_df['total_vaccine in USD_prev_1_year_million']) / merged_df['total_vaccine in USD_prev_1_year_million']
merged_df['total_immunization in USD_prev_1_year_pct_change'] = (merged_df['Immunization USD Mil'] - merged_df['total_immunization in USD_prev_1_year_million']) / merged_df['total_immunization in USD_prev_1_year_million']
merged_df['total_vaccine_delivery_cost in USD_prev_1_year_pct_change'] = (merged_df['Vaccine Delivery Cost USD Mil'] - merged_df['total_vaccine_delivery_cost in USD_prev_1_year_million']) / merged_df['total_vaccine_delivery_cost in USD_prev_1_year_million']

In [136]:
go.Figure(
    data=[go.Histogram(x=merged_df["Vaccine Delivery Cost USD Mil"], xbins={"start": -60.0, "end": 420.0, "size": 40.0})],
    layout=go.Layout(title="Histogram of Vaccine Delivery Cost USD Mil", yaxis={"title": "Count"}, bargap=0.05),
    )

In [137]:
# Step: Keep rows where Vaccine Delivery Cost USD Mil > 0
merged_df = merged_df.loc[merged_df['Vaccine Delivery Cost USD Mil'] > 0]

In [138]:
fig = px.histogram(merged_df.dropna(subset=['total_vaccine in USD_prev_1_year_pct_change']), x='Year', histfunc='avg', y='total_vaccine in USD_prev_1_year_pct_change')
fig

In [139]:
fig = px.histogram(merged_df.dropna(subset=['total_immunization in USD_prev_1_year_pct_change']), x='Year', histfunc='avg', y='total_immunization in USD_prev_1_year_pct_change')
fig

In [140]:
fig = px.histogram(merged_df.dropna(subset=['total_immunization in USD_prev_1_year_pct_change']), x='Year', histfunc='avg', y='total_immunization in USD_prev_1_year_pct_change')
fig

In [141]:

fig = px.histogram(merged_df, x='Year', histfunc='avg', y='total_vaccine_delivery_cost in USD_prev_1_year_pct_change')
fig

##### Immunization & vaccine expenditure percentage change is below 100% per year on average. However the Average vaccine delivery percentage change is over 100%. This shows us that there are data inaccuracy on the immunization and vaccine expenditure. We will exclude the data for vaccine delivery cost changes over 100%

In [142]:
# Step: Keep rows where total_vaccine_delivery_cost in USD_prev_1_year_pct_change >= 1
merged_df = merged_df.loc[merged_df['total_vaccine_delivery_cost in USD_prev_1_year_pct_change'] <= 1]

In [143]:
merged_df

           Country Region              Gavi / Income status       Year  \
196         Angola   AFRO  non-Gavi middle income countries 2007-01-01   
197        Albania   EURO  non-Gavi middle income countries 2007-01-01   
209   Burkina Faso   AFRO         Gavi low income countries 2007-01-01   
210     Bangladesh  SEARO  Gavi low-middle income countries 2007-01-01   
215        Belarus   EURO  non-Gavi middle income countries 2007-01-01   
...            ...    ...                               ...        ...   
3087      Suriname   AMRO  non-Gavi middle income countries 2021-01-01   
3095          Togo   AFRO         Gavi low income countries 2021-01-01   
3102       Tunisia   EMRO  non-Gavi middle income countries 2021-01-01   
3118        Zambia   AFRO  Gavi low-middle income countries 2021-01-01   
3119      Zimbabwe   AFRO  Gavi low-middle income countries 2021-01-01   

      total_vaccine in USD  total_immunization in USD  \
196           9.944444e+06               1.500000e+07 

### Feature Engineering

##### Create a population difference data

In [144]:
# sort the dataframe by country and year
merged_df = merged_df.sort_values(['Country', 'Year'])

# create new columns for the population percentage difference
merged_df['Population_Diff_Percent_0-1'] = None
merged_df['Population_Diff_Percent_1-5'] = None
merged_df['Population_Diff_Percent_6-10'] = None
merged_df['Population_Diff_Percent_11-15'] = None
merged_df['Population_Diff_Percent_16-20'] = None
merged_df['Population_Diff_Percent'] = None

# iterate over each country group
for country, group_df in grouped_df:
    # calculate the population difference percentage for each row
    group_df['Population_Diff_Percent_0-1'] = group_df['total_population_0-1'].pct_change()
    group_df['Population_Diff_Percent_1-5'] = group_df['total_population_1-5'].pct_change()
    group_df['Population_Diff_Percent_6-10'] = group_df['total_population_6-10'].pct_change()
    group_df['Population_Diff_Percent_11-15'] = group_df['total_population_11-15'].pct_change()
    group_df['Population_Diff_Percent_16-20'] = group_df['total_population_16-20'].pct_change()
    group_df['Population_Diff_Percent'] = group_df['total_population'].pct_change()
    # update the merged_df with the new column values
    merged_df.update(group_df)

In [145]:
# calculate the average coverage for different vaccines
merged_df['Avg_Vaccine_Coverage'] = merged_df[['BCG_COVERAGE', 'DTPCV1_COVERAGE', 'DTPCV3_COVERAGE', 'HEPB3_COVERAGE', 
                                                'HEPB_BD_COVERAGE', 'HIB3_COVERAGE', 'MCV1_COVERAGE', 'MCV2_COVERAGE',
                                                'PCV3_COVERAGE', 'POL3_COVERAGE', 'RCV1_COVERAGE', 'ROTAC_COVERAGE']].mean(axis=1)




In [146]:
# convert all columns to float data type
cols_to_float = coverage_list

merged_df[cols_to_float] = merged_df[cols_to_float].astype(float)

# create the bins and labels for the coverage categories
bins = [0, 50, 75, 90, 100]
labels = [0, 1, 2, 3]

# list of coverage columns to categorize
coverage_cols =coverage_list

# categorize each coverage column and create a new categorical column for each one
for col in coverage_cols:
    merged_df[col+'_CATEGORY'] = pd.cut(merged_df[col], bins=bins, labels=labels, include_lowest=True)


In [147]:
# # create one-hot encoding for each coverage column category
# for col in coverage_cols:
#     category_col = col + '_CATEGORY'
#     one_hot = pd.get_dummies(merged_df[category_col], prefix=category_col)
#     merged_df = pd.concat([merged_df, one_hot], axis=1)

In [148]:
# group the dataframe by country
grouped_df = merged_df.groupby('Country')

# Define a function to create the previous year columns in million
def create_prev_year_cols(col):
    return [f"{col}_prev_{i}_year" for i in range(1, 4)]
# Define a function to create the previous year columns in million

# Create new columns for 'total_vaccine in USD', 'total_immunization in USD', and 'total_vaccine_delivery_cost in USD', 
# for the value for previous year up until last 5 years, in million
for col in coverage_list:
    prev_year_cols = create_prev_year_cols(col)
    for i, prev_year_col in enumerate(prev_year_cols):
        merged_df[prev_year_col] = grouped_df[col].apply(lambda x: x.shift(i+1))
        curr_col = f"{col}_prev_1_year"
        pct_col = f"{col}_prev_{i+1}_year_pct_change"
        if i == 0:
            if curr_col in merged_df.columns and prev_year_col in merged_df.columns:
                merged_df[pct_col] = (merged_df[col] - merged_df[prev_year_col]) / merged_df[prev_year_col]
            else:
                merged_df[pct_col] = float('nan')
        else:
            if curr_col in merged_df.columns and prev_year_col in merged_df.columns:
                merged_df[pct_col] = (merged_df[curr_col] - merged_df[prev_year_col]) / merged_df[prev_year_col]
            else:
                merged_df[pct_col] = float('nan')

In [149]:
#get the next year cost
# Create new columns for the next year's values of 'total_vaccine in USD', 'total_immunization in USD', and 'total_vaccine_delivery_cost in USD'
merged_df['Next Year Vaccine USD Mil'] = grouped_df['total_vaccine in USD'].shift(-1) 
merged_df['Next Year Immunization USD Mil'] = grouped_df['total_immunization in USD'].shift(-1) 
merged_df['Next Year Vaccine Delivery Cost USD Mil'] = grouped_df['total_vaccine_delivery_cost in USD'].shift(-1) 

In [150]:
# Define a function to create the previous year columns for GDP in million
def create_prev_year_gdp_cols_million():
    return [f"GDP_prev_{i}_year" for i in range(1, 6)]

# Create new columns for GDP, for the value for previous year up until last 5 years
prev_gdp_cols = create_prev_year_gdp_cols_million()
for i, prev_gdp_col in enumerate(prev_gdp_cols):
    merged_df[prev_gdp_col] = grouped_df['GDP_Million'].apply(lambda x: x.shift(i+1))

    # Calculate the percentage change between the current year and the previous year
    curr_col = "GDP_Million"
    pct_col = f"GDP_prev_{i+1}_year_pct_change"
    if curr_col in merged_df.columns and prev_gdp_col in merged_df.columns:
        merged_df[pct_col] = (merged_df[curr_col] - merged_df[prev_gdp_col]) / merged_df[prev_gdp_col]
    else:
        merged_df[pct_col] = float('nan')


In [151]:
# sort the dataframe by country and year
merged_df.sort_values(['Country', 'Year'], inplace=True)

# group the dataframe by country and calculate the percentage change in immunization USD between consecutive years
merged_df['current_Immunization_cost_pct_change_from_last_year'] = merged_df.groupby('Country')['Immunization USD Mil'].pct_change() * 100



In [152]:
import ppscore as pps

# Compute the PPS matrix for all pairs of columns in merged_df
pps_matrix = pps.matrix(merged_df)

# Find the columns that have a PPS score of at least 0.5 with respect to the target column
target_column = 'Immunization USD Mil'
high_pps_cols = set()
num = 1
for col in merged_df.columns:

    num = num + 1
    pps_score = pps.score(merged_df, col, target_column)['ppscore']
    if (pps_score > 0.0) & (pps_score < 0.9):
        high_pps_cols.add(col)

# Print the high-PPS columns and their corresponding PPS scores
for col in high_pps_cols:
    pps_score = pps.score(merged_df, col, target_column)['ppscore']
    print(f'{col}: {pps_score}')
    
# Create a new DataFrame with only the high-PPS columns and the target column
#new_df = merged_df[high_pps_cols + [target_column]]


1
Country
2
3
4
5
total_vaccine in USD
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
GDP_Million
22
total_population
23
total_population_0-1
24
total_population_1-5
25
total_population_6-10
26
total_population_11-15
27
total_population_16-20
28
land_area
29
30
Vaccine USD Mil
31
32
33
total_vaccine in USD_prev_1_year_million
34
35
total_vaccine in USD_prev_2_year_million
36
37
total_vaccine in USD_prev_3_year_million
38
39
total_vaccine in USD_prev_4_year_million
40
41
total_vaccine in USD_prev_5_year_million
42
43
total_immunization in USD_prev_1_year_million
44
45
total_immunization in USD_prev_2_year_million
46
47
total_immunization in USD_prev_3_year_million
48
49
total_immunization in USD_prev_4_year_million
50
51
total_immunization in USD_prev_5_year_million
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120

In [153]:
high_pps_cols

{'Country',
 'GDP_Million',
 'GDP_prev_1_year',
 'GDP_prev_2_year',
 'GDP_prev_3_year',
 'GDP_prev_4_year',
 'GDP_prev_5_year',
 'Next Year Immunization USD Mil',
 'Next Year Vaccine USD Mil',
 'Vaccine USD Mil',
 'land_area',
 'total_immunization in USD_prev_1_year_million',
 'total_immunization in USD_prev_2_year_million',
 'total_immunization in USD_prev_3_year_million',
 'total_immunization in USD_prev_4_year_million',
 'total_immunization in USD_prev_5_year_million',
 'total_population',
 'total_population_0-1',
 'total_population_1-5',
 'total_population_11-15',
 'total_population_16-20',
 'total_population_6-10',
 'total_vaccine in USD',
 'total_vaccine in USD_prev_1_year_million',
 'total_vaccine in USD_prev_2_year_million',
 'total_vaccine in USD_prev_3_year_million',
 'total_vaccine in USD_prev_4_year_million',
 'total_vaccine in USD_prev_5_year_million'}

##### Extrapolate the data

In [154]:
# Specify columns to interpolate missing values
cols_to_interpolate = ['total_vaccine in USD', 'total_immunization in USD',
       'total_immunization in USD per surviving infant',
       'total_vaccine in USD per surviving infant', 'BCG_COVERAGE',
       'DTPCV1_COVERAGE', 'DTPCV3_COVERAGE', 'HEPB3_COVERAGE',
       'HEPB_BD_COVERAGE', 'HIB3_COVERAGE','MCV1_COVERAGE',
       'MCV2_COVERAGE',  'PCV3_COVERAGE', 'POL3_COVERAGE',
       'RCV1_COVERAGE', 'ROTAC_COVERAGE', 'GDP_Million', 'total_population','total_population_0-1',
       'total_population_1-5', 'total_population_6-10',
       'total_population_11-15', 'total_population_16-20', 'land_area',
       'total_vaccine_delivery_cost in USD', 'Vaccine USD Mil',
       'Immunization USD Mil', 'Vaccine Delivery Cost USD Mil',
       'total_vaccine in USD_prev_1_year_million',
       'total_vaccine in USD_prev_1_year_pct_change',
       'total_vaccine in USD_prev_2_year_million',
       'total_vaccine in USD_prev_2_year_pct_change',
       'total_vaccine in USD_prev_3_year_million',
       'total_vaccine in USD_prev_3_year_pct_change',
       'total_vaccine in USD_prev_4_year_million',
       'total_vaccine in USD_prev_4_year_pct_change',
       'total_vaccine in USD_prev_5_year_million',
       'total_vaccine in USD_prev_5_year_pct_change',
       'total_immunization in USD_prev_1_year_million',
       'total_immunization in USD_prev_1_year_pct_change',
       'total_immunization in USD_prev_2_year_million',
       'total_immunization in USD_prev_2_year_pct_change',
       'total_immunization in USD_prev_3_year_million',
       'total_immunization in USD_prev_3_year_pct_change',
       'total_immunization in USD_prev_4_year_million',
       'total_immunization in USD_prev_4_year_pct_change',
       'total_immunization in USD_prev_5_year_million',
       'total_immunization in USD_prev_5_year_pct_change',
       'total_vaccine_delivery_cost in USD_prev_1_year_million',
       'total_vaccine_delivery_cost in USD_prev_1_year_pct_change',
       'total_vaccine_delivery_cost in USD_prev_2_year_million',
       'total_vaccine_delivery_cost in USD_prev_2_year_pct_change',
       'total_vaccine_delivery_cost in USD_prev_3_year_million',
       'total_vaccine_delivery_cost in USD_prev_3_year_pct_change',
       'total_vaccine_delivery_cost in USD_prev_4_year_million',
       'total_vaccine_delivery_cost in USD_prev_4_year_pct_change',
       'total_vaccine_delivery_cost in USD_prev_5_year_million',
       'total_vaccine_delivery_cost in USD_prev_5_year_pct_change']
for j in cols_to_interpolate:
    # Group the data by country and year
    grouped = merged_df.groupby(['Country'])

    # Loop through each group
    for name, group in grouped:
        # Create a new dataframe with the year as the index
        ts = pd.DataFrame(group[j].values, index=group['Year'], columns=[j])
        # Convert the column to numeric dtype
        ts[j] = pd.to_numeric(ts[j], errors='coerce')
        # Interpolate missing values using time series method
        ts = ts.interpolate(method='time')
        # Update the merged_df with the interpolated values
        merged_df.loc[group.index,j] = ts[j].values

In [155]:
merged_df

          Country Region              Gavi / Income status       Year  \
780   Afghanistan   EMRO         Gavi low income countries 2010-01-01   
1170  Afghanistan   EMRO         Gavi low income countries 2012-01-01   
1560  Afghanistan   EMRO         Gavi low income countries 2014-01-01   
1950  Afghanistan   EMRO         Gavi low income countries 2016-01-01   
2145  Afghanistan   EMRO         Gavi low income countries 2017-01-01   
...           ...    ...                               ...        ...   
2144     Zimbabwe   AFRO  Gavi low-middle income countries 2016-01-01   
2534     Zimbabwe   AFRO  Gavi low-middle income countries 2018-01-01   
2729     Zimbabwe   AFRO  Gavi low-middle income countries 2019-01-01   
2924     Zimbabwe   AFRO  Gavi low-middle income countries 2020-01-01   
3119     Zimbabwe   AFRO  Gavi low-middle income countries 2021-01-01   

      total_vaccine in USD  total_immunization in USD  \
780            23204631.00               2.518116e+07   
1170     

In [156]:
merged_df.to_csv('immunization_prediction_data.csv')

### Data Selection

In [157]:
#merged_df = merged_df[merged_df['Year'] >= '2006']

In [158]:
selected_column = ['Year','Country',
 'GDP_Million',
 'GDP_prev_1_year',
 'GDP_prev_2_year',
 'GDP_prev_3_year',
 'Vaccine USD Mil',
 'land_area',
 'total_immunization in USD_prev_1_year_million',
 'total_immunization in USD_prev_2_year_million',
 'total_immunization in USD_prev_3_year_million',

 'total_population',
 'total_population_0-1',
 'total_population_1-5',
 'total_population_16-20',
 'total_population_6-10',
 'total_vaccine in USD',
 'total_vaccine in USD_prev_1_year_million',
 'total_vaccine in USD_prev_2_year_million',
 'total_vaccine in USD_prev_3_year_million',
 'total_vaccine_delivery_cost in USD_prev_1_year_million',
                   'Immunization USD Mil'
       ] + coverage_list


selected_data_df = merged_df[selected_column]
selected_data_df['Year'] = selected_data_df['Year'].dt.year.astype(float)
selected_data_df = selected_data_df.dropna(axis = 0)

In [159]:
selected_data_df

        Year      Country   GDP_Million  GDP_prev_1_year  GDP_prev_2_year  \
1950  2016.0  Afghanistan  18019.558182     20550.582747     20203.572960   
2145  2017.0  Afghanistan  18896.352022     18019.558182     20550.582747   
2340  2018.0  Afghanistan  18418.848300     18896.352022     18019.558182   
2926  2021.0       Angola  67404.287260     53619.073505     69309.110146   
1765  2015.0   Azerbaijan  53074.370486     69683.935845     65951.627200   
...      ...          ...           ...              ...              ...   
2144  2016.0     Zimbabwe  20548.678070     19963.120610     19091.019990   
2534  2018.0     Zimbabwe  34156.069918     20548.678070     19963.120610   
2729  2019.0     Zimbabwe  21832.234926     34156.069918     20548.678070   
2924  2020.0     Zimbabwe  21509.698406     21832.234926     34156.069918   
3119  2021.0     Zimbabwe  28371.238666     21509.698406     21832.234926   

      GDP_prev_3_year  Vaccine USD Mil  land_area  \
1950     15633.856787 

### ML

In [160]:
# Import required libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

import pandas as pd
import numpy as np

#year to start predicting
year_pred = 2020

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Define a function to fit and predict using the best model
def fit_and_predict(best_model, X_train, X_test, y_train):
    # Train the best model on the full dataset
    best_model.fit(X_train, y_train)
    
    # Make predictions on the testing set and evaluate performance
    y_pred = best_model.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return y_pred, mape, r2

# Filter data by year
train_data = selected_data_df[selected_data_df['Year'] < year_pred]
test_data = selected_data_df[selected_data_df['Year'] >= year_pred]

# Split the data into training and testing sets
target_columns = ['Immunization USD Mil']
X_col = [x for x in selected_column if x not in target_columns]
X_train = train_data[X_col].drop(['Year', 'Country'], axis=1)
X_test = test_data[X_col].drop(['Year', 'Country'], axis=1)

# Label encode categorical columns
label_encoder = LabelEncoder()
# X_train['Country'] = label_encoder.fit_transform(X_train['Country'])
# X_test['Country'] = label_encoder.transform(X_test['Country'])

# One-hot encode categorical columns (alternative to label encoding)
# ct = ColumnTransformer([('encoder', OneHotEncoder(), [0, 1, 2])], remainder='passthrough')
# X_train = ct.fit_transform(X_train)
# X_test = ct.transform(X_test)

# Define the models and their hyperparameters
models = [
    {
        'name': 'linear regression',
        'model': LinearRegression(),
        'params': {}
    },
    {
        'name': 'random forest',
        'model': RandomForestRegressor(),
        'params': {
            'model__n_estimators': [10, 50, 100, 200],
            'model__max_depth': [None, 5, 10, 20]
        }
    },
    {
        'name': 'gradient boosting',
        'model': GradientBoostingRegressor(),
        'params': {
            'model__n_estimators': [10, 50, 100, 200],
            'model__max_depth': [3, 5, 10],
            'model__learning_rate': [0.1, 0.01, 0.001]
        }
    },
    {
        'name': 'neural network',
        'model': MLPRegressor(activation='relu'),
        'params': {
            'model__hidden_layer_sizes': [(50,), (100,), (50, 50), (150, 150)],
            'model__learning_rate_init': [0.1, 0.01, 0.001],
            'model__max_iter': [500],
            'model__solver': ['adam'],
        }
    }
]

# Evaluate each model using cross-validation and select the best one for each target variable
best_models = {}
for target_column in target_columns:
    y_train = train_data[target_column]
    y_test = test_data[target_column]
    best_model = None
    best_score = None
    for model in models:
        pipeline = Pipeline([('scaler', StandardScaler()), ('model', model['model'])])
        clf = GridSearchCV(pipeline, model['params'], cv=10, scoring='neg_mean_absolute_percentage_error')
    
        clf.fit(X_train, y_train)
        score = -clf.best_score_
        print(model['name'], f'mean absolute percentage error ({target_column}):', score)
        if best_score is None or score < best_score:
            best_score = score
            best_model = clf.best_estimator_
    # Save the best model for the current target variable
    best_models[target_column] = best_model

#Get the name and parameters of the best model for each target variable
best_model_names = {target_column: best_models[target_column].named_steps['model'].__class__.__name__ for target_column in target_columns}
best_model_params = {target_column: best_models[target_column].named_steps['model'].get_params() for target_column in target_columns}
print('Best models:', best_model_names)
print('Best model parameters:', best_model_params)

for target_column in target_columns:
    best_model = best_models[target_column]
    y_pred, mape, r2 = fit_and_predict(best_model, X_train, X_test, train_data[target_column])

    # Append the predictions and error to the DataFrame
    selected_data_df[f'{target_column}_predicted'] = np.nan
    selected_data_df.loc[selected_data_df['Year'] >= year_pred, f'{target_column}_predicted'] = y_pred

selected_data_df['mape'] = selected_data_df.apply(lambda x: mean_absolute_percentage_error(x['Immunization USD Mil'], x['Immunization USD Mil_predicted']), axis=1)
selected_data_df[selected_data_df['Year'] >= year_pred].to_csv('cleaned_data_with_predictions.csv', index=False)
   


linear regression mean absolute percentage error (Immunization USD Mil): 0.4339611488040839
random forest mean absolute percentage error (Immunization USD Mil): 0.2666140452063422
gradient boosting mean absolute percentage error (Immunization USD Mil): 0.25571297880672594
neural network mean absolute percentage error (Immunization USD Mil): 0.5984529106345856
Best models: {'Immunization USD Mil': 'GradientBoostingRegressor'}
Best model parameters: {'Immunization USD Mil': {'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'squared_error', 'max_depth': 5, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200, 'n_iter_no_change': None, 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}}
